# This Notebook is an attempt at experimenting with SVG file form. 

## Goal 1. Open an .svg file and extract the paths

## Goal 2. Convert the paths into meaningful points, tracking position after each command relative to the origin

## Goal 3. Create robot commands 

In [5]:
from bs4 import BeautifulSoup # This is my personal favorite library for HTML and XML in Python
import re

# The next line opens file '76star.svg' of type 'xml'
soup = BeautifulSoup(open('76star.svg'), 'xml')
print(soup.prettify()) #bs.prettify prints a nicely formatted string, but soup is a tree

print('\n\n')
svg = soup.svg # get the svg info from soup

# finally we can find all of the path elements that are children of svg, and subsequently get the d elements iterating 
# through
for path in svg.find_all('path'): 
    print(path.get('d'))

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html>
<html>
 <body>
  <svg height="932.000000pt" preserveAspectRatio="xMidYMid meet" version="1.0" viewBox="0 0 1400.000000 932.000000" width="1400.000000pt" xmlns="http://www.w3.org/2000/svg">
   <g fill="#000000" stroke="none" transform="translate(0.000000,932.000000) scale(0.100000,-0.100000)">
    <path d="M8114 278 c-9 -42 -10 -43 -48 -40 l-39 3 31 -35 c30 -35 30 -35 17 -85 l-13 -50 30 22 30 22 25 -30 25 -30 -6 46 c-4 40 -2 49 19 69 32 30 31 34 -7 45 -29 9 -34 15 -39 52 -7 58 -14 61 -25 11z"/>
   </g>
   Sorry, your browser does not support inline SVG.
  </svg>
 </body>
</html>



M8114 278 c-9 -42 -10 -43 -48 -40 l-39 3 31 -35 c30 -35 30 -35 17 -85 l-13 -50 30 22 30 22 25 -30 25 -30 -6 46 c-4 40 -2 49 19 69 32 30 31 34 -7 45 -29 9 -34 15 -39 52 -7 58 -14 61 -25 11z



**Hopefully**, we successfully opened the '76star.svg' file and printed the data of the <path> tags. The first print statement should show the original file we worked with, while the second should show our targetted <path> data. It may not look like much, but this is nearly all we need. Unfortunately, the <path> data is not overwhelmingly beautiful, like our xml file. Below is a little more info to understand what we're really looking at



## SVG Path Info
#### source: https://css-tricks.com/svg-path-syntax-illustrated-guide/

> The <path> element is used to define a path.

> The following commands are available for path data:
```
    M = moveto
    L = lineto
    H = horizontal lineto
    V = vertical lineto
    C = curveto
    S = smooth curveto
    Q = quadratic Bézier curve
    T = smooth quadratic Bézier curveto
    A = elliptical Arc
    Z = closepath
```
>Note: All of the commands above can also be expressed with lower letters. Capital letters means absolutely positioned, lower cases means relatively positioned.

Those look like good candidates for python functions, if I've ever seen any. At this juncture, the two essential commands look to be moveto, lineto, curveto, and finally closepath. They'll probably need to take parameters, like points, cur_point, absolute=True/False. That's a problem for later though. Right now we need to get our data properly formatted. Adding some whitespace we can more easily see each individual command from our prior example 

M8114 278 

c-9 -42 -10 -43 -48 -40 

l-39 3 31 -35 

c30 -35 30 -35 17 -85 

l-13 -50 30 22 30 22 25 -30 25 -30 -6 46 

c-4 40 -2 49 19 69 32 30 31 34 -7 45 -29 9 -34 15 -39 52 -7 58 -14 61 -25 11

z

*which really means...*

**Moveto** (8114, 278) (the absolute point)

**curveto** (-9, -42) (-10, -43) (-48, -40) (relative to the current point)

**lineto** (-39, 3) (31, -35) (relative to the current point)

**curveto** (30, -35) (30, -35) (17, -85) (relative to the current point)

**lineto** (-13, -50) (30, 22) (30, 22) (25, -30) (25, -30) (-6, 46) (relative to the current point)

**curveto** (-4, 40) (-2, 49) (19, 69) (32, 30) (31, 34) (-7, 45) (-29, 9) (-34, 15) (-39, 52) (-7, 58) (-14, 61) (-25, 11) (relative to the current point)

**close path** (back to the absolute point 8114 278)

To visualize all this look at the below code, and paste it over the example https://www.w3schools.com/graphics/tryit.asp?filename=trysvg_polygon3

It is not exactly our previous example code (which is part of a larger file) but it uses the same general curves. The first path with a pink fill is the shape that our example code draws. The subsequent paths are individual parts, which make the whole, and an additional Moveto command to properly position each of them. Observe how the red curve starts at (250,250). It then moves to the relative points (-9 -42) and (-10 -43) before ending at the relative point (-48 -40), which is the absolute point (250-48, 250-40), or (202, 210). The black line starts at the absolute point (202, 210) and thus the two lines meet there. To explore this in depth, scroll to the bottom and paste that into the above link

```
<!DOCTYPE html>
<html>
<body>

<svg height="300pt" width="600pt"> 
	<g fill="pink">
      <path stroke="none" d=
          "M250 250 
          c-9 -42 -10 -43 -48 -40 
          l-39 3 31 -35 
          c30 -35 30 -35 17 -85 
          l-13 -50 30 22 30 22 25 -30 25 -30 -6 46 
          c-4 40 -2 49 19 69 32 30 31 34 -7 45 -29 9 -34 15 -39 52 -7 58 -14 61 -25 11 
          z">
  	</g> 
	<g fill="none">
      <path stroke="red" d="M 250 250
   		c-9 -42 -10 -43 -48 -40 " />
      <path stroke="black" d="M 202 210 l-39 3 31 -35" />
      <path stroke="blue" d="M 194 178 c30 -35 30 -35 17 -85" />
      <path stroke="green" d="M 211 93 l-13 -50 30 22 30 22 25 -30 25 -30 -6 46" />
      <path stroke="yellow" d="M 302 73 c-4 40 -2 49 19 69 32 30 31 34 -7 45 -29 9 -34 15 -39 52 -7 58 -14 61 -25 11" />
  	</g>  
   Sorry, your browser does not support inline SVG.
  </svg>

</body>
</html>
```

## Putting it all together

This is great to visualize and understand the data that we are working with, but now we will need to get a little more practical and create a *function*, which can
1. open an XML file
2. extract the svg path data
3. format the data into a useful format that we can create robot commands from

As the order of the paths may be important, we should use a list to store them. Each path can be further split into two distinct parts, *command* and *points*, so a tuple is natural here. As the order of the points, must also be preserved, we should store them in a list as well. Below is a visual to aid in percieving our new structure 
```
[('M', [8114, 278]),
('c', [-9, -42, -10, -43, -48, -40]),
('l', [-39, 3, 31, -35]),
('c', [30, -35, 30, -35, 17, -85]),
('l', [-13, -50, 30, 22, 30, 22, 25, -30, 25, -30, -6, 46]),
('c', [-4, 40, -2, 49, 19, 69, 32, 30, 31, 34, -7, 45, -29, 9, -34, 15, -39, 52, -7, 58, -14, 61, -25]),
('z', [])]

```

In [56]:
def svg_to_paths(file_name):
    '''
    This method takes one parameter, an xml file_name. It extracts svg tags, and returns and list containing a tuple
    with a command and a list of points.
    [(command, [point1x, point1y point2x, point2y ..., pointNx, pointNy]),]
    '''
    #load file_name and get svg tag
    soup = BeautifulSoup(open(file_name), 'xml')
    svg = soup.svg
    
    commands = {
            'M' : 'moveto',
            'L' : 'lineto',
            'H' : 'hlineto',
            'V' : 'vertical lineto',
            'C' : 'curveto',
            'S' : 'smooth curveto',
            'Q' : 'quadratic Bézier curve',
            'T' : 'smooth quadratic Bézier curveto',
            'A' : 'elliptical Arc',
            'Z' : 'closepath'
            }
    paths = []
    #pat = re.compile(r'[.]+')
    for path in svg.find_all('path'):
        #paths.append(path.get('d'))
        d = path.get('d')
        d = d.split()
        points = []
        command = ''
        for tok in d:
            if tok.isdigit():
                points.append(int(tok))
            elif tok[0] == '-':
                points.append(int(tok[1:])*(-1))
            elif tok[0].isalpha():
                tup = (command, points)
                paths.append(tup)
                command = tok[0]
                points = []
                points.append(int(tok[1:]))
            elif tok.endswith('Z') or tok.endswith('z'):
                tup = (command, points)
                paths.append(tup)
                paths.append(('z',[]))
            else:
                print(tok)
        #print(d, ',')
        del paths[0]
    return paths

triangle = svg_to_paths('76star.svg')
print('If our function works as expected, this output should resemble the format above')
for each in triangle:
    print(each)

If our function works as expected, this output should resemble the format above
('M', [8114, 278])
('c', [-9, -42, -10, -43, -48, -40])
('l', [-39, 3, 31, -35])
('c', [30, -35, 30, -35, 17, -85])
('l', [-13, -50, 30, 22, 30, 22, 25, -30, 25, -30, -6, 46])
('c', [-4, 40, -2, 49, 19, 69, 32, 30, 31, 34, -7, 45, -29, 9, -34, 15, -39, 52, -7, 58, -14, 61, -25])
('z', [])


## Goal #2
#### Convert the paths into meaningful points, tracking position after each command relative to the origin

## Goal #3
#### Create robot commands

> The following commands are available for path data:
```
    M = moveto
    L = lineto
    H = horizontal lineto
    V = vertical lineto
    C = curveto
    S = smooth curveto
    Q = quadratic Bézier curve
    T = smooth quadratic Bézier curveto
    A = elliptical Arc
    Z = closepath
```
>Note: All of the commands above can also be expressed with lower letters. Capital letters means absolutely positioned, lower cases means relatively positioned.

Those look like good candidates for python functions, if I've ever seen any. At this juncture, the two essential commands look to be moveto, lineto, curveto, and finally closepath. They'll probably need to take parameters, like points, cur_point, absolute=True/False. That's a problem for later though. Right now we need to get our data properly formatted. Adding some whitespace we can more easily see each individual command from our prior example 

## Explore .svg More
#### paste this code into

```
<!DOCTYPE html>
<html>
<body>

<svg height="300pt" width="600pt"> 

     <!-- This is the example shape @(250,250) instead of (8114, 278) -->
	<g fill="pink">
      <path stroke="none" d="M250 250 c-9 -42 -10 -43 -48 -40 l-39 3 31 -35 c30 -35 30 -35 17 -85 l-13 -50 30 22 30 22 25 -30 25 -30 -6 46 c-4 40 -2 49 19 69 32 30 31 34 -7 45 -29 9 -34 15 -39 52 -7 58 -14 61 -25 11 z"/>
  	</g> 
    
    <!-- These are the individual part making the shape -->
	<g fill="none">
      <path stroke="red" d="M 250 250
   		c-9 -42 -10 -43 -48 -40 " />
      <path stroke="black" d="M 202 210 l-39 3 31 -35" />
      <path stroke="blue" d="M 194 178 c30 -35 30 -35 17 -85" />
      <path stroke="green" d="M 211 93 l-13 -50 30 22 30 22 25 -30 25 -30 -6 46" />
      <path stroke="yellow" d="M 302 73 c-4 40 -2 49 19 69 32 30 31 34 -7 45 -29 9 -34 15 -39 52 -7 58 -14 61 -25 11" />
  	</g>  
    
     <!-- Mark relevant points -->
     
     <!-- Below we can observe how curveto and lineto frustratingly differ-->
     <!-- lineto moves relative to the previous point in the command-->
     
     <!-- curveto has two type of points, which I'll call anchors and control -->
     <!-- the first and last points in curve to are anchors -->
     <!-- controls moves relative to the most recent anchor-->
     <!-- there seem to alway be two -->
     <!-- then there is another anchor, which also moves relative to the last anchor-->
     
     
    <g fill="red"> <!-- red curveto points -->
      <circle id="A" cx="250" cy="250" r="2" />
      <!-- NOTE: These are control points -->
      <circle fill="white" stroke="red" id="B" cx="241" cy="209" r="2" />
      <circle fill="white" stroke="red" id="C" cx="240" cy="207" r="2" />
      <circle id="D" cx="202" cy="210" r="4" />
    </g>
    <g fill="Black"> <!-- black lineto points -->
      <circle id="A" cx="202" cy="210" r="2" />
      <circle id="B" cx="163" cy="213" r="2" />
      <circle id="C" cx="194" cy="178" r="4" />
    </g>
    <g fill="Blue"> <!-- blue curve to points -->
      <circle id="A" cx="194" cy="178" r="2" />
      <!-- NOTE: These are control points at the same location-->
      <circle fill="white" stroke="blue" id="B" cx="224" cy="143" r="2" />
      <circle fill="white" stroke="blue" id="C" cx="224" cy="143" r="2" />
      <circle id="D" cx="211" cy="93" r="4" />
    </g>
    <g fill="green"> <!-- green lineto points -->
      <circle id="A" cx="211" cy="93" r="2" />
      <circle id="B" cx="198" cy="43" r="2" /> <!---13 -50-->
      <circle id="C" cx="228" cy="65" r="2" /> <!--30 22-->
      <circle id="D" cx="258" cy="87" r="2" /> <!-- 30 22 -->
      <circle id="E" cx="283" cy="57" r="2" /> <!--25 -30 --> 
      <circle id="F" cx="308" cy="27" r="2" /> <!-- 25 -30 --> 
      <circle id="G" cx="302" cy="73" r="4" /> <!-- -6 46 -->
    </g>
    <g fill="Yellow"> <!-- yellow curve to points -->
      <circle id="A" cx="302" cy="73" r="2" />
      <!--  Two control points -->
      <circle fill="black" stroke="yellow" id="B" cx="298" cy="113" r="2" />
      <circle fill="black" stroke="yellow" id="C" cx="300" cy="122" r="2" />
      <!-- new anchor -->
      <circle fill="yellow" id="D" cx="321" cy="142" r="2" />
      <!--  Two more control points -->
      <circle fill="black" stroke="yellow" id="E" cx="353" cy="172" r="2" />
      <circle fill="black" stroke="yellow" id="F" cx="352" cy="176" r="2" />
      <!-- new anchor -->
      <circle fill="yellow" id="D" cx="314" cy="187" r="2" />
      <!--  Two more control points -->
      <circle fill="black" stroke="yellow" id="E" cx="285" cy="198" r="2" />
      <circle fill="black" stroke="yellow" id="F" cx="280" cy="202" r="2" />
      <!-- new anchor -->
      <circle fill="yellow" id="D" cx="275" cy="239" r="2" />
      <!--  Two more control points -->
      <circle fill="black" stroke="yellow" id="E" cx="268" cy="297" r="2" />
      <circle fill="black" stroke="yellow" id="F" cx="261" cy="300" r="2" />
      <!--  Final Anchor / End Point / Closes loop -->
      <circle id="x" cx="250" cy="250" r="4" />
      
      <!-- -7 58 -14 61 -25 11 -->
    </g>
   Sorry, your browser does not support inline SVG.
  </svg>

</body>
</html>
```